In [1]:
import streamlit as st
import pandas as pd
import requests
import numpy as np
import yfinance as yf
from dune_client.client import DuneClient

In [2]:
@st.cache_data()
def fetch_data_from_api(api_url, params=None):
    response = requests.get(api_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if 'rows' in data['result']:
            return pd.DataFrame(data['result']['rows'])
        return data
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return pd.DataFrame()  # or an empty dict

2024-03-27 10:39:00.064 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [3]:
def fetch_historical_data(api_url, api_key):
    # Use the API key either as a query parameter or in the headers
    params = {'vs_currency': 'usd', 'days': 'max', 'interval': 'daily', 'x_cg_demo_api_key': api_key}
    headers = {'x-cg-demo-api-key': api_key}  # Alternatively, use this header

    response = requests.get(api_url, params=params, headers=headers)

    if response.status_code == 200:
        # Parse the JSON response
        historical_pricedata = response.json()
        # Extract the 'prices' and 'market_caps' data
        historical_price = historical_pricedata['prices']
        market_cap = pd.DataFrame(historical_pricedata['market_caps'], columns=['date', 'marketcap'])

        # Convert the 'timestamp' column from UNIX timestamps in milliseconds to datetime objects
        history = pd.DataFrame(historical_price, columns=['timestamp', 'price'])
        history['date'] = pd.to_datetime(history['timestamp'], unit='ms')
        history.set_index('date', inplace=True)
        history.drop(columns='timestamp', inplace=True)

        vol = pd.DataFrame(historical_pricedata['total_volumes'], columns=['date', 'volume'])
        vol['date'] = pd.to_datetime(vol['date'], unit='ms')
        vol.set_index('date', inplace=True)
        
        return mkr_history, mkr_market_cap, mkr_vol
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return pd.DataFrame(), pd.DataFrame()

In [4]:
api_key_dune = st.secrets["api_key"]
api_key_cg = st.secrets["api_key_cg"]
api_key_FRED = st.secrets["FRED_API_KEY"]

In [5]:
dune = DuneClient(api_key_dune)

In [6]:
def fetch_dune_data(num):
    result = dune.get_latest_result(num)
    return pd.DataFrame(result.result.rows)

In [18]:
pd.options.display.float_format = '{:,.2f}'.format

### First, lets get MakerDAO Financial Statements from https://dune.com/steakhouse/makerdao

Balance Sheet

In [7]:
# Balance Sheet

bs_raw = dune.get_latest_result(2840463)

In [8]:
bs_df = pd.DataFrame(bs_raw.result.rows)
bs_df['period'] = pd.to_datetime(bs_df['period'])
bs_df.set_index('period', inplace=True)
bs_df.index = bs_df.index.normalize()
bs_df = bs_df.sort_index()

In [10]:
bs_df

,balance,item,normalized
period,,,
2020-07-01 00:00:00+00:00,NaN,Others assets,NaN
2020-07-01 00:00:00+00:00,"-200,158.70",Equity,-0.00
2020-07-01 00:00:00+00:00,"144,805,383.35",Crypto-Loans,1.00
2020-07-01 00:00:00+00:00,"-111,909,502.58",DAI,-0.77
2020-07-01 00:00:00+00:00,NaN,Stablecoins,NaN
...,...,...,...
2024-03-21 00:00:00+00:00,"2,358,355,394.97",Crypto-Loans,0.50
2024-03-21 00:00:00+00:00,"-1,384,700,394.26",DSR,-0.30
2024-03-21 00:00:00+00:00,"1,232,860,684.32",Stablecoins,0.26


In [ ]:
bs_path = '../data/csv/bs.csv'
bs_df.to_csv(bs_path)

In [19]:
bs_csv = pd.read_csv(bs_path, index_col='period', parse_dates=True)

In [20]:
#categorizing items as asset, liability, or equity
def categorize_item(item):
    if item in ['Crypto-Loans', 'Real-World Assets', 'Others assets', 'Stablecoins']:
        return 'Assets'
    elif item in ['DAI','DSR']:  # Assuming DAI represents a liability here; adjust according to your accounting rules
        return 'Liabilities'
    elif item == 'Equity':
        return 'Equity'
    else:
        return 'Other'  # For any item not explicitly categorized

# Assuming 'df' is your DataFrame
bs_csv['category'] = bs_csv['item'].apply(categorize_item)



In [21]:
bs_csv = bs_csv.iloc[::-1]

In [22]:
bs_csv

,balance,item,normalized,category
period,,,,
2024-03-21 00:00:00+00:00,"-67,156,270.08",Equity,-0.01,Equity
2024-03-21 00:00:00+00:00,"-3,223,114,058.66",DAI,-0.69,Liabilities
2024-03-21 00:00:00+00:00,"1,232,860,684.32",Stablecoins,0.26,Assets
2024-03-21 00:00:00+00:00,"-1,384,700,394.26",DSR,-0.30,Liabilities
2024-03-21 00:00:00+00:00,"2,358,355,394.97",Crypto-Loans,0.50,Assets
...,...,...,...,...
2020-07-01 00:00:00+00:00,NaN,Stablecoins,NaN,Assets
2020-07-01 00:00:00+00:00,"-111,909,502.58",DAI,-0.77,Liabilities
2020-07-01 00:00:00+00:00,"144,805,383.35",Crypto-Loans,1.00,Assets


MONTHLY Income Statement/PnL (also includes more detailed balance sheet)

In [18]:
is_df = fetch_dune_data(2641549) 

2024-03-21 12:12:27.070 
  command:

    streamlit run C:\Users\brandyns pc\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-03-21 12:12:27.071 No runtime found, using MemoryCacheStorageManager


In [19]:
is_df.head()

,expenses,item,lending_income,liquidation_income,month,net_income,period,trading_income,value,year
0,"-3,857,879.46",1 - PnL,"11,054,226.90","4,623.88",3,"7,200,971.32",2024-03,0.00,NaN,2024
1,NaN,1.1 - Lending Revenues,NaN,NaN,3,NaN,2024-03,NaN,"18,130,771.84",2024
2,NaN,1.2 - Liquidations Revenues,NaN,NaN,3,NaN,2024-03,NaN,"4,623.88",2024
3,NaN,1.3 - Trading Revenues,NaN,NaN,3,NaN,2024-03,NaN,0.00,2024
4,NaN,1.4 - Lending Expenses,NaN,NaN,3,NaN,2024-03,NaN,"-7,076,544.94",2024


In [20]:
is_df_wide = is_df.pivot_table(index='period', columns='item', values='value', aggfunc='sum').reset_index()
is_df_wide = is_df_wide.iloc[::-1]

In [21]:
is_df_wide.head()

item,period,1 - PnL,1.1 - Lending Revenues,1.2 - Liquidations Revenues,1.3 - Trading Revenues,1.4 - Lending Expenses,1.5 - Liquidations Expenses,1.6 - Workforce Expenses,1.9 - Net Income,2 - Assets,2.1 - Crypto Loans,2.2 - Trading Assets,2.8 - Operating Reserves,2.9 - Total Assets,3 - Liabilities & Equity,3.1 - Liabilities (DAI),3.7 - Equity (Surplus Buffer),3.8 - Equity (Operating Reserves),3.9 - Total Liabilities & Equity
52,2024-03,0.00,"18,130,771.84","4,623.88",0.00,"-7,076,544.94",0.00,"-3,857,879.46","7,200,971.32",0.00,"3,442,061,505.58","1,232,860,684.32",0.00,"4,674,922,189.91",0.00,"4,607,814,452.91","67,107,736.99",0.00,"4,674,922,189.91"
51,2024-02,0.00,"29,691,662.12",0.00,0.00,"-4,567,709.04",0.00,"-1,581,115.17","23,542,837.91",0.00,"4,083,073,663.21","913,091,407.04",0.00,"4,996,165,070.25",0.00,"4,929,564,735.87","66,600,334.38",0.00,"4,996,165,070.25"
50,2024-01,0.00,"20,751,255.62","27,376.30",0.00,"-5,943,924.23",0.00,"-5,669,555.33","9,165,152.35",0.00,"4,321,343,136.29","563,679,392.72",0.00,"4,885,022,529.00",0.00,"4,832,474,047.47","52,548,481.53",0.00,"4,885,022,529.00"
49,2023-12,0.00,"14,270,261.48",0.00,0.00,"-6,484,200.84",0.00,"-2,658,930.56","5,127,130.08",0.00,"4,820,477,732.35","400,493,643.71",0.00,"5,220,971,376.06",0.00,"5,168,577,458.38","52,393,917.68",0.00,"5,220,971,376.06"
48,2023-11,0.00,"29,666,040.79",455.03,0.00,"-6,376,789.60",0.00,"-2,974,302.25","20,315,403.96",0.00,"4,636,257,421.47","707,478,489.39",0.00,"5,343,735,910.86",0.00,"5,285,580,181.04","58,155,729.83",0.00,"5,343,735,910.86"


In [24]:
is_path = '../data/csv/is.csv'

In [ ]:
is_df.to_csv(is_path)

In [25]:
is_csv = pd.read_csv(is_path, index_col='period', parse_dates=True)

In [26]:
is_csv.head()

,Unnamed: 0,expenses,item,lending_income,liquidation_income,month,net_income,trading_income,value,year
period,,,,,,,,,,
2024-03-01,0,"-3,857,879.46",1 - PnL,"11,054,226.90","4,623.88",3,"7,200,971.32",0.00,NaN,2024
2024-03-01,1,NaN,1.1 - Lending Revenues,NaN,NaN,3,NaN,NaN,"18,130,771.84",2024
2024-03-01,2,NaN,1.2 - Liquidations Revenues,NaN,NaN,3,NaN,NaN,"4,623.88",2024
2024-03-01,3,NaN,1.3 - Trading Revenues,NaN,NaN,3,NaN,NaN,0.00,2024
2024-03-01,4,NaN,1.4 - Lending Expenses,NaN,NaN,3,NaN,NaN,"-7,076,544.94",2024


Assets/Revenue Per Type
Coinbase asset type: http://forum.makerdao.com/t/mip81-coinbase-usdc-institutional-rewards/17703/254?u=sebventures


In [26]:
assets_raw = dune.get_latest_result(58495)

In [27]:
assets_p_t_ts = pd.DataFrame(assets_raw.result.rows)
assets_p_t_ts['dt'] = pd.to_datetime(assets_p_t_ts['dt'])
assets_p_t_ts.set_index('dt', inplace=True)

In [28]:
assets_p_t_ts.head()

,annual_revenues,asset,blended_rate,collateral,revenues,total_annual_revenues,total_asset,total_blended_rate
dt,,,,,,,,
2024-03-21 00:00:00+00:00,"1,930,000.00","772,000,000.00",0.00,Coinbase,NaN,"240,716,327.65","4,692,077,547.49",0.05
2024-03-21 00:00:00+00:00,"155,559,844.17","1,002,509,890.98",0.16,ETH,"250,862.26","240,716,327.65","4,692,077,547.49",0.05
2024-03-21 00:00:00+00:00,NaN,"1,022,233,852.61",NaN,Lending Protocols,0.00,"240,716,327.65","4,692,077,547.49",0.05
2024-03-21 00:00:00+00:00,"47,213.19","93,864,104.51",0.00,Liquidity Pools,NaN,"240,716,327.65","4,692,077,547.49",0.05
2024-03-21 00:00:00+00:00,-0.00,0.00,-1.00,Others,NaN,"240,716,327.65","4,692,077,547.49",0.05


In [32]:
assets_p_t_ts.to_csv(as_path)

In [27]:
as_path = '../data/csv/as.csv'
as_csv = pd.read_csv(as_path, index_col='dt', parse_dates=True)

Daily Interest Revenues By Vault

In [28]:
ir_v = fetch_dune_data(3567939) 

In [29]:
ir_v['period'] = pd.to_datetime(ir_v['period'])
ir_v.set_index('period', inplace=True)

In [30]:
ir_v.head()

,collateral,daily_revenues
period,,
2024-03-27,ETH-A,"21,754.48"
2024-03-27,ETH-B,"56,751.53"
2024-03-27,ETH-C,"103,498.46"
2024-03-27,WBTC-A,"11,273.70"
2024-03-27,WBTC-B,"25,135.47"


In [31]:
daily_int_path = '../data/csv/d_int.csv'
ir_v.to_csv(daily_int_path)
ir_csv = pd.read_csv(daily_int_path, index_col='period', parse_dates=True)

In [32]:
ir_csv

,collateral,daily_revenues
period,,
2024-03-27,ETH-A,"21,754.48"
2024-03-27,ETH-B,"56,751.53"
2024-03-27,ETH-C,"103,498.46"
2024-03-27,WBTC-A,"11,273.70"
2024-03-27,WBTC-B,"25,135.47"
...,...,...
2019-11-15,BAT-A,0.00
2019-11-15,ETH-A,0.00
2019-11-13,BAT-A,0.00


Dai Maturity Profile

Step-by-Step Process:
Tracking DAI Movements:

First, all transactions involving DAI are tracked to understand how DAI moves in and out of wallets. This includes both inflows (adding DAI to a wallet) and outflows (removing DAI from a wallet).
Defining Maturity Buckets:

Maturity buckets are predefined categories based on time durations, such as "1-day", "1-week", "1-month", "1-year", etc. Each bucket represents a hypothesis about how long DAI tends to stay put before being moved again.
Assigning Weights to Buckets:

Weights are assigned to each maturity bucket to reflect assumptions or historical observations about the distribution of DAI across these buckets. For example, if historically 30% of DAI is moved or used within a day, then the "1-day" bucket might get a weight of 0.30 (or 30%).
Applying Weights Based on Wallet Types:

DAI can be held in different types of wallets or contracts, each with its own expected behavior. For example, DAI in a savings contract (like DSR) might be considered more long-term ("1-year"), while DAI in a regular wallet might be more liquid ("1-day" or "1-week"). The weights applied to the DAI in these wallets reflect these expectations.
Calculating DAI Amounts per Bucket:

For each wallet (or DAI holding), the total amount of DAI is distributed across the maturity buckets based on the assigned weights. This means if a wallet has 100 DAI and the "1-day" bucket weight is 30%, then 30 DAI is considered to have a 1-day maturity.
The process is repeated for each wallet and each maturity bucket, based on the specific weights for that wallet type and the total DAI it holds.
Aggregating Across the Ecosystem:

Finally, to get the ecosystem-wide view, the amounts of DAI in each maturity bucket from all wallets are aggregated. This provides a snapshot of how much DAI is considered to be in each maturity bucket across the entire MakerDAO system at any given time.

In [39]:
d_m = fetch_dune_data(907852)

In [40]:
d_m['dt'] = pd.to_datetime(d_m['dt'])

In [41]:
d_m

,dt,maturity,outflow,outflow_dai_only,outflow_surplus_buffer,total_period
0,2024-03-20 00:00:00+00:00,1-week,"589,090,233.01","589,090,233.01",0.00,"4,661,750,574.18"
1,2024-03-20 00:00:00+00:00,1-month,"135,641,898.57","135,641,898.57",0.00,"4,661,750,574.18"
2,2024-03-20 00:00:00+00:00,1-block,"451,479,837.89","451,479,837.89",0.00,"4,661,750,574.18"
3,2024-03-20 00:00:00+00:00,1-year,"2,833,931,375.49","2,766,819,019.01","67,112,356.48","4,661,750,574.18"
4,2024-03-20 00:00:00+00:00,1-day,"451,479,837.89","451,479,837.89",0.00,"4,661,750,574.18"
...,...,...,...,...,...,...
9523,2019-11-13 00:00:00+00:00,1-week,0.00,0.00,NaN,0.00
9524,2019-11-13 00:00:00+00:00,1-block,0.00,0.00,NaN,0.00
9525,2019-11-13 00:00:00+00:00,1-month,0.00,0.00,NaN,0.00
9526,2019-11-13 00:00:00+00:00,1-day,0.00,0.00,NaN,0.00


In [42]:

d_m.to_csv(d_m_path)


In [33]:
d_m_path = '../data/csv/d_m.csv'
d_m_csv = pd.read_csv(d_m_path, index_col='dt', parse_dates=True)

In [34]:
d_m_csv

,Unnamed: 0,maturity,outflow,outflow_dai_only,outflow_surplus_buffer,total_period
dt,,,,,,
2024-03-20 00:00:00+00:00,0,1-week,"589,090,233.01","589,090,233.01",0.00,"4,661,750,574.18"
2024-03-20 00:00:00+00:00,1,1-month,"135,641,898.57","135,641,898.57",0.00,"4,661,750,574.18"
2024-03-20 00:00:00+00:00,2,1-block,"451,479,837.89","451,479,837.89",0.00,"4,661,750,574.18"
2024-03-20 00:00:00+00:00,3,1-year,"2,833,931,375.49","2,766,819,019.01","67,112,356.48","4,661,750,574.18"
2024-03-20 00:00:00+00:00,4,1-day,"451,479,837.89","451,479,837.89",0.00,"4,661,750,574.18"
...,...,...,...,...,...,...
2019-11-13 00:00:00+00:00,9523,1-week,0.00,0.00,NaN,0.00
2019-11-13 00:00:00+00:00,9524,1-block,0.00,0.00,NaN,0.00
2019-11-13 00:00:00+00:00,9525,1-month,0.00,0.00,NaN,0.00


MakerDAO Stablecoin Ratio
This can give insights into the proportion of assets held in stablecoins (including DAI) relative to other assets. A higher stablecoin ratio might suggest a preference for stability within the MakerDAO system, which can have implications for DAI's stability.

In [44]:
stablecoin_ratio_df = fetch_dune_data(58136)

In [45]:
stablecoin_ratio_df['dt'] = pd.to_datetime(stablecoin_ratio_df['dt'])
stablecoin_ratio_df.set_index('dt', inplace=True)

,,assets_size,assets_stablecoins,assets_usdc,dt,stablecoins_ratio,usdc_ratio
0,1.0,"4,692,077,547.49","510,492,745.88","509,792,133.98",2024-03-21T00:00:00Z,0.11,0.11
1,1.0,"4,665,516,334.77","451,367,053.87","450,666,441.97",2024-03-20T00:00:00Z,0.10,0.10
2,1.0,"4,703,718,034.84","404,362,171.82","403,655,435.55",2024-03-19T00:00:00Z,0.09,0.09
3,1.0,"4,744,355,701.71","399,931,319.66","399,224,583.39",2024-03-18T00:00:00Z,0.08,0.08
4,1.0,"4,699,947,545.51","632,391,001.19","631,684,264.92",2024-03-17T00:00:00Z,0.13,0.13
...,...,...,...,...,...,...,...
1581,1.0,"15,769,739.59",NaN,NaN,2019-11-22T00:00:00Z,NaN,NaN
1582,1.0,"13,612,364.30",NaN,NaN,2019-11-21T00:00:00Z,NaN,NaN
1583,1.0,"9,808,793.17",NaN,NaN,2019-11-20T00:00:00Z,NaN,NaN
1584,1.0,"7,455,988.46",NaN,NaN,2019-11-19T00:00:00Z,NaN,NaN


In [46]:

stablecoin_ratio_df.to_csv(stablecoin_ratio_path)


In [35]:
stablecoin_ratio_path = '../data/csv/srp.csv'
stablecoin_ratio_csv = pd.read_csv(stablecoin_ratio_path, index_col='dt', parse_dates=True)

In [36]:
stablecoin_ratio_csv

,Unnamed: 0,,assets_size,assets_stablecoins,assets_usdc,stablecoins_ratio,usdc_ratio
dt,,,,,,,
2024-03-21 00:00:00+00:00,0,1.00,"4,692,077,547.49","510,492,745.88","509,792,133.98",0.11,0.11
2024-03-20 00:00:00+00:00,1,1.00,"4,665,516,334.77","451,367,053.87","450,666,441.97",0.10,0.10
2024-03-19 00:00:00+00:00,2,1.00,"4,703,718,034.84","404,362,171.82","403,655,435.55",0.09,0.09
2024-03-18 00:00:00+00:00,3,1.00,"4,744,355,701.71","399,931,319.66","399,224,583.39",0.08,0.08
2024-03-17 00:00:00+00:00,4,1.00,"4,699,947,545.51","632,391,001.19","631,684,264.92",0.13,0.13
...,...,...,...,...,...,...,...
2019-11-22 00:00:00+00:00,1581,1.00,"15,769,739.59",NaN,NaN,NaN,NaN
2019-11-21 00:00:00+00:00,1582,1.00,"13,612,364.30",NaN,NaN,NaN,NaN
2019-11-20 00:00:00+00:00,1583,1.00,"9,808,793.17",NaN,NaN,NaN,NaN


Maker Peg Stability Module Stats
Data on the Peg Stability Module, which helps maintain DAI's peg to the USD, can be vital. Insights into the inflows, outflows, and balances within the PSM can directly indicate efforts to stabilize DAI.

In [48]:
psm_stats_df = fetch_dune_data(17216)

In [50]:
psm_stats_df['date'] = pd.to_datetime(psm_stats_df['date'])

In [51]:

psm_stats_df.to_csv(psm_stats_path)


In [37]:
psm_stats_path = '../data/csv/psm.csv'
psm_stats_csv = pd.read_csv(psm_stats_path, index_col='date', parse_dates=True)

In [38]:
psm_stats_csv

,Unnamed: 0,change,change_excl_rwa,change_excl_rwa_30d_avg,change_excl_rwa_7d_avg,fees,inflow,inflow_exl_rwa,lifetime_fees,lifetime_turnover,outflow,psm_balance,turnover
date,,,,,,,,,,,,,
2024-03-21 00:00:00+00:00,0,"59,125,692.01","59,125,692.01","-39,765,372.80","15,853,783.28",0.00,"138,429,104.70","138,429,104.70","4,449,255.59","100,329,058,375.20","79,303,412.69","510,492,745.88","217,732,517.40"
2024-03-20 00:00:00+00:00,1,"47,004,882.05","47,004,882.05","-43,689,564.71","6,842,814.63",0.00,"114,637,705.11","114,637,705.11","4,449,255.59","100,111,325,857.80","67,632,823.07","451,367,053.87","182,270,528.18"
2024-03-19 00:00:00+00:00,2,"4,430,852.16","4,430,852.16","-43,887,466.96","-60,382,354.15",0.00,"191,755,292.35","191,755,292.35","4,449,255.59","99,929,055,329.62","187,324,440.20","404,362,171.82","379,079,732.55"
2024-03-18 00:00:00+00:00,3,"-232,459,681.52","-232,459,681.52","-44,026,058.34","-66,040,075.59",0.00,"144,111,445.81","144,111,445.81","4,449,255.59","99,549,975,597.08","376,571,127.34","399,931,319.66","520,682,573.15"
2024-03-17 00:00:00+00:00,4,"64,591,276.21","64,591,276.21","-36,201,438.41","-15,168,476.40",0.00,"85,172,448.88","85,172,448.88","4,449,255.59","99,029,293,023.93","20,581,172.67","632,391,001.19","105,753,621.55"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-02 00:00:00+00:00,1174,NaN,NaN,1.00,1.00,NaN,NaN,NaN,0.01,1.00,NaN,1.00,NaN
2021-01-01 00:00:00+00:00,1175,NaN,NaN,1.00,1.00,NaN,NaN,NaN,0.01,1.00,NaN,1.00,NaN
2020-12-31 00:00:00+00:00,1176,NaN,NaN,1.00,1.00,NaN,NaN,NaN,0.01,1.00,NaN,1.00,NaN


Where is dai lets us know how dai is being used; dai in lending could be considered to add to stability 

In [53]:
where_is_dai_df = fetch_dune_data(54599)

In [54]:
where_is_dai_df['dt'] = pd.to_datetime(where_is_dai_df['dt'])

,balance,dt,total_balance,wallet
0,"29,833,280.83",2024-03-21 00:00:00.000 UTC,"4,626,344,329.30",CeFi
1,"316,083,654.94",2024-03-21 00:00:00.000 UTC,"4,626,344,329.30",Other
2,"2,312,432,867.95",2024-03-21 00:00:00.000 UTC,"4,626,344,329.30",EOA
3,"346,715,770.42",2024-03-21 00:00:00.000 UTC,"4,626,344,329.30",Bridge
4,"719,771.00",2024-03-21 00:00:00.000 UTC,"4,626,344,329.30",Treasury
...,...,...,...,...
11107,"8,997,146.49",2020-06-02 00:00:00.000 UTC,"120,016,193.71",Dai Savings
11108,"6,212,584.87",2020-06-02 00:00:00.000 UTC,"120,016,193.71",Lending
11109,181.37,2020-06-02 00:00:00.000 UTC,"120,016,193.71",Bridge
11110,"8,670,808.34",2020-06-02 00:00:00.000 UTC,"120,016,193.71",Other


In [67]:
wid_path = '../data/csv/wid.csv'
where_is_dai_df.to_csv(wid_path)
where_is_dai_csv = pd.read_csv(wid_path, index_col='dt', parse_dates=True)

In [56]:
where_is_dai_csv

,Unnamed: 0,balance,total_balance,wallet
dt,,,,
2024-03-21 00:00:00+00:00,0,"29,833,280.83","4,626,344,329.30",CeFi
2024-03-21 00:00:00+00:00,1,"316,083,654.94","4,626,344,329.30",Other
2024-03-21 00:00:00+00:00,2,"2,312,432,867.95","4,626,344,329.30",EOA
2024-03-21 00:00:00+00:00,3,"346,715,770.42","4,626,344,329.30",Bridge
2024-03-21 00:00:00+00:00,4,"719,771.00","4,626,344,329.30",Treasury
...,...,...,...,...
2020-06-02 00:00:00+00:00,11107,"8,997,146.49","120,016,193.71",Dai Savings
2020-06-02 00:00:00+00:00,11108,"6,212,584.87","120,016,193.71",Lending
2020-06-02 00:00:00+00:00,11109,181.37,"120,016,193.71",Bridge


Daily surplus buffer
Provides information on the surplus buffer in MakerDAO, which is a key financial metric. The surplus buffer acts as a reserve to cover potential system shortfalls and ensures the stability and solvency of the system. This data could be valuable for understanding the financial health and risk management strategies of MakerDAO over time

In [57]:
daily_surplus_buffer = fetch_dune_data(482305)

In [59]:
daily_surplus_buffer['period'] = pd.to_datetime(daily_surplus_buffer['period'])

In [68]:
dsb_path = '../data/csv/dsb.csv'
daily_surplus_buffer.to_csv(dsb_path)
daily_surplus_buffer_csv = pd.read_csv(dsb_path, index_col='period', parse_dates=True)

In [62]:
daily_surplus_buffer_csv

,Unnamed: 0,delta_30d,delta_90d,surplus_buffer
period,,,,
2024-03-21 00:00:00+00:00,0,"52,271,732.38","-16,001,514.72","46,221,726.29"
2024-03-20 00:00:00+00:00,1,"52,720,064.25","-15,733,901.00","46,587,588.92"
2024-03-19 00:00:00+00:00,2,"74,342,616.64","-16,769,602.06","46,876,134.92"
2024-03-18 00:00:00+00:00,3,"74,766,083.21","-14,805,925.67","47,172,545.34"
2024-03-17 00:00:00+00:00,4,"101,243,056.00","-5,693,850.31","49,662,241.06"
...,...,...,...,...
2023-03-27 00:00:00+00:00,360,"-26,383,772.01","-10,987,865.84","72,251,278.84"
2023-03-26 00:00:00+00:00,361,"-27,630,052.02","-11,211,009.97","72,215,544.47"
2023-03-25 00:00:00+00:00,362,"-35,785,812.51","-11,332,382.02","72,156,132.21"


Stability Fee history and Vault History - Rates Set by DAO
dart = debt balance

In [78]:
sf_df = fetch_dune_data(3549361)

In [80]:
sf_df['period'] = pd.to_datetime(sf_df['period'])

In [84]:
sf_path = '../data/csv/sf.csv'
sf_df.to_csv(sf_path)
sf_history_csv = pd.read_csv(sf_path, index_col='period', parse_dates=True)

In [85]:
sf_history_csv

,Unnamed: 0,annualized,annualized_revenues,dart,ilk,total_ann_revenues
period,,,,,,
2024-03-21 00:00:00+00:00,0,0.05,0.00,0.00,GNO-A,"176,643,660.32"
2024-03-21 00:00:00+00:00,1,0.00,"42,545.93","70,910,068.44",GUNIV3DAIUSDC2-A,"176,643,660.32"
2024-03-21 00:00:00+00:00,2,0.03,-0.00,-0.00,LINK-A,"176,643,660.32"
2024-03-21 00:00:00+00:00,3,0.01,-0.00,-0.00,YFI-A,"176,643,660.32"
2024-03-21 00:00:00+00:00,4,0.01,0.00,0.00,COMP-A,"176,643,660.32"
...,...,...,...,...,...,...
2019-11-01 00:00:00+00:00,95411,NaN,NaN,NaN,WBTC-A,NaN
2019-11-01 00:00:00+00:00,95412,NaN,NaN,NaN,UNIV2UNIETH-A,NaN
2019-11-01 00:00:00+00:00,95413,NaN,NaN,NaN,RWA005-A,NaN


DAI Savings Rate Historical - Set by DAO

In [86]:
dsr_rate = fetch_dune_data(2721037)

In [91]:
dsr_rate_path = '../data/csv/dsr.csv'
dsr_rate.to_csv(dsr_rate_path)
dsr_rate_csv = pd.read_csv(dsr_rate_path, index_col='dt', parse_dates=True)

In [93]:
dsr_rate_csv['dsr_rate'].describe()

count   444.00
mean      0.04
std       0.03
min       0.00
25%       0.01
50%       0.05
75%       0.05
max       0.15
Name: dsr_rate, dtype: float64

dsr flows

In [ ]:
dsr_flows = fetch_dune_data(1753750)

### Now for CoinGecko Crypto Market Data

In [ ]:
#lets get price feeds for accepted collateral types

ir_v['collateral'].unique()

In [ ]:
eth_historical_api = "https://api.coingecko.com/api/v3/coins/ethereum/market_chart"
wsteth_historical_api = "https://api.coingecko.com/api/v3/coins/wrapped-steth/market_chart"
wbtc_historical_api = "https://api.coingecko.com/api/v3/coins/wrapped-bitcoin/market_chart"

In [ ]:
eth_history, eth_historical_mk, eth_vol = fetch_historical_data(eth_historical_api, api_key_cg)
wsteth_history, wsteth_historical_mk, wsteth_vol = fetch_historical_data(wsteth_historical_api, api_key_cg)
wbtc_history, wbtc_historical_mk, wbtc_vol = fetch_historical_data(wbtc_historical_api, api_key_cg)

In [ ]:
wbtc_vol

In [ ]:
wsteth_history

In [ ]:
wbtc_history

In [ ]:
# DAI Price feed and vol

dai_historical_api = "https://api.coingecko.com/api/v3/coins/dai/market_chart"

dai_history, dai_historical_mk, dai_vol = fetch_historical_data(dai_historical_api, api_key_cg)

In [ ]:
dai_vol